In [137]:
import os
from pathlib import Path

import cv2
import numpy as np

In [138]:
cwd = Path.cwd().as_posix()
path_10m = os.path.join(cwd, 'data/apriltags_run2/din2_tags/10m')
paths_10m = list(Path(path_10m).glob("*.tif"))
path_20m = os.path.join(cwd, 'data/apriltags_run2/din2_tags/20m')
paths_20m = list(Path(path_20m).glob("*.tif"))
path_40m = os.path.join(cwd, 'data/apriltags_run2/din2_tags/40m')
paths_40m = list(Path(path_40m).glob("*.tif"))

In [154]:
horizontal_focal_length_pixels = 1581.7867974691412
horizontal_focal_center_pixels = 678.6724626822399
vertical_focal_length_pixels = 1581.7867974691412
vertical_focal_center_pixels = 529.4318832108801
tag_size = 0.38

In [155]:
from typing import List
from robotpy_apriltag import AprilTagPoseEstimator, AprilTagDetector

pose_estimator = AprilTagPoseEstimator(
    AprilTagPoseEstimator.Config(tag_size, horizontal_focal_length_pixels, vertical_focal_length_pixels,
                                 horizontal_focal_center_pixels, vertical_focal_center_pixels))


def verify_detections(tag) -> bool:
    valid_ids = [4]
    return tag.getId() in valid_ids


families = [
    "tagCircle21h7"
]
detectors: List[AprilTagDetector] = []
for family in families:
    d = AprilTagDetector()
    d.addFamily(family)
    config = AprilTagDetector.Config()
    config.quadDecimate = 1.0
    config.numThreads = 4
    config.refineEdges = 1.0
    d.setConfig(config)
    detectors.append(d)


def post_estimate_tags(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    tags = [detector.detect(img) for detector in detectors]
    tags = [item for row in tags for item in row if verify_detections(item)]
    return [pose_estimator.estimate(tag).z for tag in tags]

In [156]:
altitudes_10m = [post_estimate_tags(path.as_posix()) for path in paths_10m]
altitudes_20m = [post_estimate_tags(path.as_posix()) for path in paths_20m]
altitudes_40m = [post_estimate_tags(path.as_posix()) for path in paths_40m]

In [157]:
altitudes_10m = [[int(alt) for alt in alts] for alts in altitudes_10m]
altitudes_20m = [[int(alt) for alt in alts] for alts in altitudes_20m]
altitudes_40m = [[int(alt) for alt in alts] for alts in altitudes_40m]

In [158]:
altitudes_10m_flat = [item for row in altitudes_10m for item in row]
altitudes_20m_flat = [item for row in altitudes_20m for item in row]
altitudes_40m_flat = [item for row in altitudes_40m for item in row]

In [159]:
altitudes_40m_flat.sort()
altitudes_40m_flat

[9,
 13,
 15,
 16,
 18,
 19,
 21,
 23,
 24,
 24,
 24,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 29,
 29,
 29,
 34,
 34,
 35,
 35,
 35,
 35,
 36,
 39,
 40,
 40,
 44,
 44,
 44,
 45,
 45,
 46,
 47,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 49,
 49,
 49,
 49,
 49,
 49]

In [160]:
print("Mean:")
print("10m",np.mean(altitudes_10m_flat))
print("20m",np.mean(altitudes_20m_flat))
print("40m",np.mean(altitudes_40m_flat))
print("Median:")
print("10m",np.median(altitudes_10m_flat))
print("20m",np.median(altitudes_20m_flat))
print("40m",np.median(altitudes_40m_flat))


Mean:
10m 10.618421052631579
20m 19.815217391304348
40m 36.594202898550726
Median:
10m 11.0
20m 21.0
40m 40.0
